# NLP con librería SPACY

Al igual que NLTK, spaCy es una librería de procesamiento de lenguaje natural (PLN) para Python de código abierto.    
**spaCy** ofrece a los usuarios un conjunto de herramientas y modelos para realizar diversas tareas relacionadas con el procesamiento y análisis de texto.    
Las principales características de **spaCy** son:   

1. **Tokenización**: puede dividir texto en unidades más pequeñas llamadas “tokens”, que pueden ser palabras o partes de palabras.
2. **Etiquetado gramatical**: contiene modelos entrenados que pueden etiquetar cada token en un texto con información gramatical, como partes del discurso y etiquetas sintácticas.
3. **Reconocimiento de entidades nombradas**: permite identificar y clasificar entidades nombradas en un texto, como nombres de personas, organizaciones, lugares, fechas, cantidades, entre otros.
4. **Análisis sintáctico**: facilita la realización de análisis sintáctico de las oraciones para capturar la estructura gramatical y las dependencias entre las palabras.
5. **Lematización**: reduce las palabras a su forma base (lemas), lo que ayuda a normalizar el texto y simplificar el análisis.
6. **Modelos pre-entrenados**: proporciona modelos pre-entrenados para varios idiomas que pueden utilizarse para realizar tareas de PLN sin la necesidad de entrenar un modelo desde cero. Estos modelos incluyen información gramatical, reconocimiento de entidades y análisis sintáctico.
7. **Eficiencia y velocidad**: es una librería rápida y eficiente, lo que la hace adecuada para el procesamiento de grandes volúmenes de texto.

## Análisis de sentimientos en español con spaCy.

Los pasos para realizar análisis de sentimiento en español con spaCy son los mismos que se realizan con NLTK en inglés.    
Obviamente, las funciones que se deben usar son diferentes. En concreto, los pasos que se deben seguir son:   

- preprocesamiento de datos, 
- extracción de características, 
- entrenamiento del modelo y 
- clasificación de nuevos textos.

### 1. Preprocesamiento de datos   

El procesamiento de datos en spaCy es ligeramente más sencillo en spaCy que en NLTK.    
En este caso concreto, se deben importar los modelos entrenados que se han descargado antes y usarlos para obtener los tokens ya tematizados y en minúsculas.    
Este proceso se realiza en el siguiente código:

In [1]:
# A instalar desde la terminal:
# pip install spacy
# python -m spacy download es_core_news_sm

import spacy

text = "Me encanta el contenido del blog de Inteligencia Artificial, los artículos son fantásticos."

nlp = spacy.load('es_core_news_sm')
doc = nlp(text)

# Eliminación de palabras irrelevantes (stopwords) y signos de puntuación
tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]

# Reconstrucción del texto preprocesado
preprocessed_text = ' '.join(tokens)
preprocessed_text

'encantar contenido blog inteligencia artificial artículo fantástico'

En este bloque de código se cargan los modelos mediante la función ```spacy.load()``` (si se ha descargado otro se deberá reemplazar el nombre en el parámetro de la función por el que se desee usar).    
Al modelo importado se le puede pasar la cadena de texto para obtener los tokens (```nlp(text)```).    
Posteriormente se filtran aquellos que no son ni stopwords ni elementos de puntuación (```not token.is_stop and not token.is_punct```) para, en el mismo paso, ***lematizar*** y convertir en minúsculas (```token.lemma_.lower()```). El resultado es el listado de tokens procesados.

Se puede observar como la herramienta usa como token el **infinitivo del verbo** en lugar de su versión conjugada (encantar en lugar de encanta). También **los términos en plural aparecen en singular** (artículo en lugar de artículos). Todo esto facilita el análisis de los textos.

### 2. Extracción de características

En spaCy, la extracción de características también se puede hacer de una forma sencilla. Para ello, se debe iterar sobre los tokens y crear una lista con el conteo de términos.   

Una posible opción para hacer esto es la que se muestra en el siguiente código de ejemplo:


In [2]:
features = {}
doc = nlp(preprocessed_text)

for token in doc:
    if not token.is_stop and not token.is_punct:
        if token.lemma_.lower() in features:
            features[token.lemma_.lower()] += 1
        else:
            features[token.lemma_.lower()] = 1
            
features

{'encantar': 1,
 'contenido': 1,
 'blog': 1,
 'inteligencia': 1,
 'artificial': 1,
 'artículo': 1,
 'fantástico': 1}

Esto genera un diccionario donde la palabra es la clave y el valor es el número de ocurrencias de esta en el texto.

### 3. Conjunto de datos de entrenamiento y factorización de los datos.   

Antes de llevar a cabo un análisis de sentimientos en español con spaCy, es necesario disponer de un conjunto de datos para el entrenamiento. Para ello se recurre a una traducción de un conjunto de datos definido por nosotros y que realizamos en el bloque de código siguiente:

In [3]:
training_data = [
    ("Me encanta el contenido del blog de Inteligencia Artificial, los artículos son fantásticos.", "positivo"),
    ("El código no funciona, me ha dado un error al ejecutarlo.", "negativo"),
    ("Me encanta este producto.", "positivo"),
    ("Esta película fue terrible.", "negativo"),
    ("El clima está agradable hoy.", "positivo"),
    ("Me siento triste por las noticias.", "negativo"),
    ("Es solo un libro promedio.", "neutral")
]

También se puede optar por crear funciones con las que se factorizan los pasos vistos en los apartados anteriores, de forma que el código final sea más modular.   

Por ejemplo:  

In [4]:
import spacy

def preprocess_text(text):
    """
    Realiza el preprocesamiento básico de un texto en idioma español utilizando spaCy.

    Args:
        text (str): El texto a ser preprocesado.

    Returns:
        str: El texto preprocesado.
    """
    nlp = spacy.load('es_core_news_sm')
    doc = nlp(text)

    # Eliminación de palabras irrelevantes (stopwords) y signos de puntuación
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]

    # Reconstrucción del texto preprocesado
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


def extract_features(text):
    """
    Extrae las características del texto utilizando spaCy y devuelve un diccionario de características.

    Args:
        text (str): El texto del cual extraer características.

    Returns:
        dict: Un diccionario que representa las características extraídas del texto.
    """
    features = {}
    doc = nlp(text)
    for token in doc:
        if not token.is_stop and not token.is_punct:
            if token.lemma_.lower() in features:
                features[token.lemma_.lower()] += 1
            else:
                features[token.lemma_.lower()] = 1
    return features

### 4. Entrenamiento del modelo
Para el análisis de sentimientos, uno de los modelos/algoritmos que mejor funciona es **Naive Bayes**. A diferencia de NLTK, spaCy no cuenta con una implementación propia, pero se puede optar por la de que se dispone en Scikit-learn.    
Por tanto, para entrenar el modelo solamente sería necesario preprocesar los datos, extraer las características y crear un conjunto de entrenamiento para el **modelo MultinomialNB() (Multinomial Naive Bayes)**.    

Los pasos que a implementar son:

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB

# Preprocesamiento de los datos de entrenamiento
preprocessed_training_data = [(preprocess_text(text), label) for text, label in training_data]

# Extracción de características de los datos de entrenamiento
training_features = [extract_features(text) for text, _ in preprocessed_training_data]
vectorizer = DictVectorizer(sparse=False)
X_train = vectorizer.fit_transform(training_features)

# Etiquetas de los datos de entrenamiento
y_train = [label for _, label in preprocessed_training_data]

# Entrenamiento del clasificador Naive Bayes
classifier = MultinomialNB()
_ = classifier.fit(X_train, y_train)

### 5. Clasificación de nuevos textos

Una vez entrenado el modelo, este se puede usar para predecir el sentimiento de nuevos textos.    
Simplemente se repiten con los nuevos textos las transformaciones realizadas sobre el conjunto de entrenamiento y el resultado se le pasa al modelo.

In [6]:
# Nuevo texto para clasificar
new_text = "Me encantó mucho del concierto."

# Preprocesamiento del nuevo texto
preprocessed_text = preprocess_text(new_text)

# Extracción de características del nuevo texto
features = extract_features(preprocessed_text)
X_test = vectorizer.transform([features])

# Clasificación del nuevo texto
sentiment = classifier.predict(X_test)
print("Sentimiento:", sentiment[0])

Sentimiento: positivo


## Conclusiones.   


**spaCy** es una librería alternativa a **NLTK** con la que también se puede realizar análisis de sentimientos. Contando con la ventaja de que también se puede hacer en español y otros idiomas gracias a los modelos pre-entrenados que se pueden descargar.    
Esto también simplifica el trabajo con la librería. Por eso, en el caso de querer realizar análisis de sentimientos en español, spaCy es una de las opciones que siempre se debe tener en cuenta.

### *Anexo*. Detección de entidades nombradas (Named Entity Recognition NER)   

Esta funcionalidad nos permite tokenizar el texto e identificar palabras clave, extrayendo información que indica y clasifica en categorías como organizaciones, lugares, cantidades,...

El código siguiente muestra un pequeño ejemplo de ello, haciendo uso de un complemento de la propia librería `spaCy` denominado [`displaCy`](https://demos.explosion.ai/displacy):

In [2]:
from spacy import displacy
from collections import Counter
import es_core_news_sm
nlp = es_core_news_sm.load()
doc = nlp("Me encanta el viajar por Castilla y León, sobretodo por la provincia de Valladolid.")
print([(X.text, X.label_) for X in doc.ents])

[('Castilla y León', 'LOC'), ('provincia de Valladolid', 'LOC')]


In [11]:
# spacy.displacy.serve(doc, style="ent") # Para visualizar en un navegador web local
# Se ha modificado el fichero devcontainer.json para que el puerto 5001 esté disponible!!!
spacy.displacy.serve(doc, style="ent", port=5001, host='0.0.0.0') # Para visualizar en un navegador web externo


Using the 'ent' visualizer
Serving on http://0.0.0.0:5001 ...



172.17.0.1 - - [16/Mar/2025 18:09:24] "GET / HTTP/1.1" 200 1112
172.17.0.1 - - [16/Mar/2025 18:09:24] "GET /favicon.ico HTTP/1.1" 200 1112


Shutting down server on port 5001.


In [ ]:
# Renderizado directo sobre Jupyter Notebook
displacy.render(nlp(str(doc)), jupyter=True, style='ent')

In [5]:
displacy.render(nlp(str(doc)), style='dep', jupyter = True, options = {'distance': 80})

A continuación, lematizamos esta frase.

In [6]:
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(str(doc)) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]

[('encanta', 'VERB', 'encantar'),
 ('viajar', 'VERB', 'viajar'),
 ('Castilla', 'PROPN', 'Castilla'),
 ('León', 'PROPN', 'León'),
 ('sobretodo', 'NOUN', 'sobretodo'),
 ('provincia', 'NOUN', 'provincia'),
 ('Valladolid', 'PROPN', 'Valladolid')]

Finalmente, usando un diccionario, extraemos las N.E.R.

In [7]:
dict([(str(x), x.label_) for x in nlp(str(doc)).ents])

{'Castilla y León': 'LOC', 'provincia de Valladolid': 'LOC'}

##  Anexo. Similitud de textos.   

Determinados modelos han sido entrenados de manera que las palabras se etiquetan aludiendo a conceptos semánticos. Por ejemplo, un perro es un mamífero y, a su vez, un animal. Un etiquetado correcto nos situa en lugares más próximos entre sí a un perro y a un gato (dos mamíferos) que a un perro y a un salmón. De igual forma, una manzana es a la vez fruta y comida, aunque la manzana se parece más a una naranja que a un filete de ternera, tratándose en ambos casos de comida.     

Usando técnicas de vectorización y cálculo, se puede analizar dos textos y establecer la similitud que tienen entre sí, aunque obviamente sea algo muy "subjetivo" y dependiente del dataset usado en el entrenamiento.

El siguiente es un ejemplo ilustrativo de un cálculo de similitud mediante la librería `spaCy`, usando  el diccionario español usado en este mismo notebook en los ejemplos anteriores.

In [11]:
#python -m spacy download es_core_news_lg
import es_core_news_lg
import spacy
from spacy import displacy
from collections import Counter

nlp = es_core_news_lg.load()
doc1 = nlp("Carlos se come una manzana")
doc2 = nlp("Maria se come una ensalada")
doc3 = nlp("Maria y Carlos se comen una pizza")
doc4 = nlp("Maria y Carlos ven una película")


In [12]:
# Similaridad entre dos documentos
print(doc1, "<->", doc2, " --> ", doc1.similarity(doc2))
print(doc1, "<->", doc3, " --> ", doc1.similarity(doc3))
print(doc1, "<->", doc4, " --> ", doc1.similarity(doc4))
print(doc2, "<->", doc3, " --> ", doc2.similarity(doc3))
print(doc2, "<->", doc4, " --> ", doc2.similarity(doc4))
print(doc1, "<->", doc4, " --> ", doc1.similarity(doc4))

Carlos se come una manzana <-> Maria se come una ensalada  -->  0.9487316608428955
Carlos se come una manzana <-> Maria y Carlos se comen una pizza  -->  0.8953322172164917
Carlos se come una manzana <-> Maria y Carlos ven una película  -->  0.34944161772727966
Maria se come una ensalada <-> Maria y Carlos se comen una pizza  -->  0.9006785154342651
Maria se come una ensalada <-> Maria y Carlos ven una película  -->  0.3849201202392578
Carlos se come una manzana <-> Maria y Carlos ven una película  -->  0.34944161772727966


Observamos distintos niveles de similitud, en función de la "actividad" realizada (verbo) y como, incluso cambiando el sujeto, también nos marca distinto valor en acciones similares.

Enlace de interés para comprender visualmente esta similitud y como funcionan las técnicas de [`word embbeding`](https://projector.tensorflow.org/)